# Projeto 2 - NLP

-----

Nome:  Johnny Hideki Horita <br>
Turma: 780

Os segundo projeto do módulo de Machine Learning será focado no processamento de linguagem natural! Usaremos os algoritmos aprendidos e as técnicas vistas na segunda parte do curso para extrairmos informações relevantes de texto. Mais precisamente, de publicações no Twitter.

## Os Dados

Utilizaremos um Dataset obtido do Twitter com 100K postagens entre os dias 01/08/2018 e 20/10/2018. Cada postagem é classificada como **positiva**, **negativa** ou **neutra**.  

Dois arquivos serão disponilizados para o desenvolvimento dos modelos, um para treino/validação e outro para submissão. Os arquivos se encontram na pasta */Dados/train* e */Dados/subm*, respectivamente.

Descrição das colunas:

- **id**: ID único para o tweet  
- **tweet_text**: Texto da publicação no Twitter  
- **tweet_date**: Data da publicação no Twitter  
- **sentiment**: 0, se negativo; 1, se positivo; 2, se neutro  
- **query_used**: Filtro utilizado para buscar a publicação

## O Problema

Você deverá desenvolver um modelo para detectar o sentimento de uma publicação do Twitter a classificando em uma das três categorias: **positiva**, **negativa** ou **neutra**. O texto da publicação está disponível na coluna "tweet_text". Teste pelo menos 3 técnicas de NLP diferentes e escolha a métrica de avaliação que julgar mais pertinente.  

Escolha o melhor modelo e gere uma base a partir dos dados de submissão, que estão no caminho ```Dados/subm/Subm3Classes.csv```, com o seguinte formato:


|id|sentiment_predict
|-|-|
|12123232|0
|323212|1
|342235|2

Salve essa tabela como um arquivo csv com o nome ```<nome>_<sobrenome>_nlp_degree.csv``` e submeta-o como parte da entrega final do projeto.  

Para ajudar no desenvolvimento, é possível dividir o projeto em algumas fases:

- **Análise de consistência dos dados**: analise se os dados estão fazendo sentido, se os campos estão completos e se há dados duplicados ou faltantes. Se julgar necessário, trate-os.    


- **Análise exploratória**: analise a sua base como um todo, verifique o balanceamento entre as classes e foque, principalmente, na coluna ```tweet_text```.    


- **Pré-processamento e transformações**: projetos de NLP exigem um considerável pré-processamento. Foque no tratamento da string do texto. Procure começar com tratamentos simples e adicione complexidade gradualmente. Nessa etapa você testará diferentes técnicas de transformações, como o Bag Of Words e o TF-IDF.    


- **Treinamento do modelo**: depois das transformações, você poderá executar o treinamento do modelo classificador. Nessa etapa o problema se torna semelhante aos abordados na primeira parte do módulo. Você pode testar diversos classificadores como RandomForest, AdaBoost, entre outros. Otimize os hiperparâmetros do modelo com técnicas como a GridSearch e a RandomizedSearch.    


- **Conclusões**: descreva, em texto, as conclusões sobre os seus estudos. O modelo é capaz de identificar o sentimento das publicações? É possível extrapolar o modelo para outros contextos, como a análise de sentimento de uma frase qualquer? Pense em questões pertinentes e relevantes que você tenha obtido durante o desenvolvimento do projeto!     



## Critérios de avaliação

Os seguintes itens serão avaliados:

1. Desenvolvimento das etapas descritas acima;


2. Reprodutibilidade do código: seu código será executado e precisa gerar os mesmos resultados apresentados por você;


3. Clareza: seu código precisa ser claro e deve existir uma linha de raciocínio direta. Comente o código em pontos que julgar necessário para o entendimento total;


4. Justificativa das conclusões obitdas: não existirá certo ou errado, mas as decisões e as conclusões precisam ser bem justificadas com base nos resultados obtidos.  

O desempenho do modelo **não** será considerado como critério de avaliação.  

## Informações gerais

- O projeto deve ser desenvolvido individualmente;


- Data de divulgação: 11/01/2022;


- Aula de monitoria: 19/01/2022;


- Data de entrega: 26/01/2022;


- Entrega através do Class: Árvore de Decisão -> Exercícios -> Projeto 2


Anexar, na entrega, o notebook de desenvolvimento e o arquivo .csv de submissão, da seguinte forma:  

notebook: ```<nome>_<sobrenome>_<númeroTurma>_projeto_2.ipynb```   
csv: ```<nome>_<sobrenome>_<númeroTurma>_projeto_2_submissao.csv```


## Dicas

### Base de treino e submissão

A base de submissão não possui a variável de saída, portanto ela será utilizada **apenas** para gerar o arquivo que acompanha a submissão do projeto.      

### Tente encontrar possíveis vieses

É muito comum que modelos de NLP possuam fortes vieses, como a tendência de relacionar palavras específicas com alguma classe de saída. Tente encontrar vieses no seu estudo, isso pode ajudar a tirar boas conclusões. o campo "query_used" pode ser útil para essa análise.  

### O pré-processamento é a chave para um bom desempenho

Essa é a etapa que mais vai contribuir para o desempenho do seu modelo. Seja criativo e desenvolva essa etapa de uma maneira que seja fácil de aplicar o mesmo processamento para uma nova base, você terá que fazer isso para gerar a base de submissão.

### Um termômetro para o seu desenvolvimento

Após a correção do seu projeto, o professor irá disponibilizar a sua acurácia obtida na base de submissão. Você pode interpretar esse resultado como a simulação do resultado do seu modelo em produção. Uma diferença entre o resultado do estudo e o resultado de submissão indica um grau de **overfitting** no seu modelo.

-------

# Desenvolvimento do projeto

## 1. Análise de consistência dos dados

### Importando bibliotecas

In [1]:
%%time

# Bibliotecas
import os
import re
import string
import random

import pandas as pd
import numpy as np
import datetime
import requests
import warnings
from collections import Counter
from pprint import pprint

from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import squarify
import plotly.offline as py
import plotly_express as px
from plotly import graph_objs as go
import plotly.figure_factory as ff

from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from mlxtend.plotting import plot_confusion_matrix

from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score, StratifiedKFold, cross_validate

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Logistic Regression
from sklearn.linear_model import LogisticRegression 
# Support Vector Machine
from sklearn.svm import SVC 
# Naive Bayes (Gaussian, Multinomial)
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
# Stochastic Gradient Descent Classifier
from sklearn.linear_model import SGDClassifier
# KNN (k-nearest neighbor)
from sklearn.neighbors import KNeighborsClassifier
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
# Random Forest
from sklearn.ensemble import RandomForestClassifier
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
# XGBoost Classifier
from xgboost import XGBClassifier
# LGBM Classifier
from lightgbm import LGBMClassifier
# Ada Boosting Classifier
from sklearn.ensemble import AdaBoostClassifier
# Dummy Boosting Classifier
from sklearn.dummy import DummyClassifier

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from unidecode import unidecode

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#!pip install enelvo
from enelvo.normaliser import Normaliser

# Abaixo seguem 2 formas para a instalação do spaCy: via conda ou pip
# Instalação utilizando conda
#!conda install -c conda-forge spacy

# Instalação utilizando Pip
#!pip install -U pip setuptools wheel
#!pip install -U spacy

import spacy
from spacy.util import compounding
from spacy.util import minibatch

from tqdm import tqdm

import shap

import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from IPython.core.display import HTML as Center
from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

print('Bibliotecas carregadas com sucesso.')

Bibliotecas carregadas com sucesso.
Wall time: 8.27 s


### Padrões

In [2]:
# Definição de padrões para gráficos e cores

sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

colors = sns.color_palette("pastel") # deep, pastel, Set1 Set2 Set3, icefire, tab10, muted, colorlind, coolwarm
cmap_colors = 'GnBu'

font_path = "./fonts/CabinSketch-Bold.ttf"

In [3]:
# Definição de padrões para centralização de gráficos no notebook

Center(""" <style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style> """)

In [4]:
# Definição de variáveis auxiliares

rollback = 0

### Funções

In [5]:
# Função de avaliação dos valores de NaN no dataframe

def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Valores faltantes', 1 : '% de Valores Totais'})
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% de Valores Totais', ascending=False).round(1)
        print ("O dataframe tem " + str(df.shape[1]) + " colunas e " + str(df.shape[0]) + " linhas.\n"      
            "Existem " + str(mz_table.shape[0]) +
              " colunas que têm valores faltantes.")
        mz_table.to_excel('missing_and_zero_values.xlsx', freeze_panes=(1,0), index = True)
        return mz_table


In [6]:
# Função de avaliação de modelos de aprendizagem de máquinas

def test_models_plot_roc_auc_curve(model_list, col_model_name, col_model, X_train, X_test, y_train, y_test):
    plt.figure(figsize=(15,7))
    for mdl in model_list:
        model = mdl[col_model]
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        y_predproba = model.predict_proba(X_test)[:,1]

        #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predproba)
        #auc = metrics.roc_auc_score(y_test, y_predict)

        fpr, tpr, thresholds = metrics.accuracy_curve(y_test, y_predproba)        
        auc = metrics.accuracy_score(y_test, y_predict)
        
        plt.plot(fpr, tpr, label='%s ROC (AUC = %0.4f)' % (mdl[col_model_name], auc))
        print("Model      : %s" % mdl[col_model_name])
        calc_predict(mdl[col_model_name], y_test, y_predict)
        
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC-AUC curve')
    plt.legend(loc="lower right")
    plt.show()

In [7]:
# Função de avaliação de modelos apresentação da matriz de confusão

def test_models_plot_confusion_matrix(model_list, col_model_name, col_model, X_train, X_test, y_train, y_test):
    for mdl in model_list:
        model = mdl[col_model]
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        
        print("")        
        print("=" * 55)        
        print("Model      : %s" % mdl[col_model_name])
        calc_predict(mdl[col_model_name], y_test, y_predict)
        
        cm = confusion_matrix(y_test, y_predict)
        cm_matrix = pd.DataFrame(data=cm, columns=['Atual Positivo:1', 'Atual Negativo:0'], 
                                         index=['Pred. Positivo:1', 'Pred. Negativo:0'])

        print('Matriz Confusão\n\n', cm)
        print('\nV. Positivos(VP) = ', cm[0,0])
        print('V. Negativos(VN) = ', cm[1,1])
        print('F. Positivos(FP) = ', cm[0,1])
        print('F. Negativos(FN) = ', cm[1,0])

        sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')
        plt.show()


In [8]:
# Função para cálcular as métricas 

def calc_predict(col_model_name, y_test, y_predict):
    #print("ROC - AUC  : %0.4f " % metrics.roc_auc_score(y_test, y_predict))
    print("Accuracy   : %0.4f " %  accuracy_score(y_test, y_predict))
    print("Precision  : %0.4f " % precision_score(y_test, y_predict, average='weighted'))
    print("Recall     : %0.4f " % recall_score(y_test, y_predict, average='weighted'))
    print("F1 - Score : %0.4f " % f1_score(y_test, y_predict, average='weighted'))
    print("MAE        : %0.4f " % mean_absolute_error(y_test, y_predict))
    print("RMSE       : %0.4f " % np.sqrt(mean_squared_error(y_test, y_predict)))
    print("R2         : %0.4f " % r2_score(y_test, y_predict))
    print("")
    print(classification_report(y_test, y_predict))
    print("=" * 55)
    print("")


In [9]:
# Função para calcular a importância da variável no modelo

rcParams['figure.figsize'] = 12, 4

def modelfit(alg, dtrain, predictors, target, performCV=True, printFeatureImportance=True, cv_folds=5):
    # Adequando as classes para treino
    alg.fit(dtrain[predictors], dtrain[target])
        
    # Previsão de saída para o conjunto de dados de teste
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    # Utilizando o Cross Validation
    if performCV:
        cv_score = cross_val_score(alg, dtrain[predictors], dtrain[target], cv=cv_folds, scoring='roc_auc')
    
    #Exibindo relatório:
    print (f"\nRelatório do Modelo {alg}")
    print ("\nAcuracia : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    #print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predictions))
    
    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
    #Exibindo gráfico da importancia das variáveis
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Importância das variáveis', color=colors)
        plt.ylabel('Pontuação')


In [10]:
# Função para geração de cores

def random_colours(number_of_colors):
    '''
    Função simples para geração de cores aleatórias.
    Entrada:
        número_de_cores - valor inteiro indicando o número de cores que vão ser geradas.
    Saída:
        Cor no seguinte formato: ['#E86DA4'].
        
    '''
    colors = []
    for i in range(number_of_colors):
        colors.append("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))
    return colors


In [11]:
# Função para contagem de palavras 

def words_unique(sentiment, numwords, raw_words):
    '''
    Entrada:
        segmento - Categoria do segmento (ex. 2 = 'Neutro');
        numwords - quantas palavras específicas se pretende ver no resultado final; 
        raw_words - lista do texto;
        
    Resultado: 
        dataframe com informação sobre a palavra específica e quantas vezes aparece no texto (por ordem decrescente com base nas suas contagens).

    '''
    allother = []
    for item in dfc[dfc.sentiment != sentiment]['list_words']:
        for word in item:
            allother .append(word)
    allother  = list(set(allother ))
    
    specificnonly = [x for x in raw_text if x not in allother]
    
    mycounter = Counter()
    
    for item in dfc[dfc.sentiment == sentiment]['list_words']:
        for word in item:
            mycounter[word] += 1
    keep = list(specificnonly)
    
    for word in list(mycounter):
        if word not in keep:
            del mycounter[word]
    
    Unique_words = pd.DataFrame(mycounter.most_common(numwords), columns = ['words','count'])
    
    return Unique_words


In [12]:
# Abaixo seguem 2 formas para a instalação do spaCy: via conda ou pip

# Instalação utilizando conda
#!conda install -c conda-forge spacy

# Instalação utilizando Pip
#!pip install -U pip setuptools wheel
#!pip install -U spacy

# Bilbioteca em portugues
# Efficiency
#!python -m spacy download pt_core_news_sm

# Accuracy
#!python -m spacy download pt_core_news_lg

spc_pt = spacy.load('pt_core_news_lg')
nltk.download('stopwords')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('portuguese')
#Adicionando stopwords que não estão na lista do nltk 
stopwords.append("'")
stopwords.append("pra")
stopwords.append("tá")
stopwords.append("tão")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnny.horita\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnny.horita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
# Função para tratamento da variável de texto para definir melhores valores para classificação da modelagem
# e ou normalizar o tratamento da variável de texto utilizando a biblioteca enelvo

# instanciando
normalizador = Normaliser(tokenizer='readable', sanitize=True, capitalize_acs=True, capitalize_pns=True) # capitalize_inis=True

def nlp_tratar_texto(texto, normalize=False):
    #Remover endereços de sites
    texto_sem_url = re.sub(r'https?:\/\/\S+', '', texto)
        
    #Remover e-mail / users
    #texto_sem_email = re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*\.?[A-Za-z0-9]*', '', texto_sem_url)
    texto_sem_email = re.sub(r'[A-Za-z0-9]*@\S+', '', texto_sem_url)
    
    if normalize==True:
        # Tratamento do texto utilizando o enelvo (Normalize)
        texto_norm = normalizador.normalise(texto_sem_email)
    
        #Remover caracteres que não são letras e tokenização
        texto_tratado =  re.findall(r'\b[A-zÀ-úü]+\b', texto_norm.lower())
    else:
        texto_tratado =  re.findall(r'\b[A-zÀ-úü]+\b', texto_sem_email.lower())

    #Remover stopwords
    stop = set(stopwords)
    palavras = [w for w in texto_tratado if w not in stop]
    palavras_string = " ".join(palavras)

    #Instanciar o objeto spacy
    spc_letras =  spc_pt(palavras_string)

    #Lemmização 
    tokens = [token.lemma_ if token.pos_ == 'VERB' else str(token) for token in spc_letras]

    #problemas com verbo ir
    ir = ['vou', 'vais', 'vai', 'vamos', 'ides', 'vão']
    tokens = ['ir' if token in ir else str(token) for token in tokens]
    
    return tokens

In [14]:
# Função para apresentar nuvem de palavras

def plot_wordcloud(text, title = None, backcolor = 'white', clrmap = ''):

    wordcloud = WordCloud(
                            background_color=backcolor,
                            width=2000, 
                            height=800,
                            colormap=clrmap, 
                            font_path=font_path, 
                            collocations = False)

    wordcloud.generate(text)
    
    fig = plt.figure(figsize=(12,10))

    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(title)
    plt.show()


### Inicializando Dataframe

In [15]:
# Importando arquivo

df = pd.read_csv('./dados/train/Train3Classes.csv')


In [16]:
# Quantidade de linhas e colunas

qtl, qtc = df.shape

# Quantidade de linhas duplicadas

qtd, _ = df[df.duplicated(keep=False)].shape

print(f'Quantidade de linhas...........: {qtl}')
print(f'Quantidade de linhas duplicadas: {qtd}')
print(f'Quantidade de colunas..........: {qtc}')


Quantidade de linhas...........: 95000
Quantidade de linhas duplicadas: 0
Quantidade de colunas..........: 5


In [17]:
# Informações do dataframe

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95000 entries, 0 to 94999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          95000 non-null  int64 
 1   tweet_text  95000 non-null  object
 2   tweet_date  95000 non-null  object
 3   sentiment   95000 non-null  int64 
 4   query_used  95000 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.6+ MB


In [18]:
# Avaliando os valores nulos do dataframe

missing_values_table(df)


O dataframe tem 5 colunas e 95000 linhas.
Existem 0 colunas que têm valores faltantes.


,Valores faltantes,% de Valores Totais,Data Type



---

**Conclusões:** 

O dataframe é composto por 05 colunas e 95.000 registros.

A tabela acima NÃO apresenta valores faltantes.



In [19]:
# Lista de colunas do dataframe

df.columns


Index(['id', 'tweet_text', 'tweet_date', 'sentiment', 'query_used'], dtype='object')

In [20]:
# Listagem das primeiras linhas do dataframe

df.head()


,id,tweet_text,tweet_date,sentiment,query_used
0,1049721159292346368,Rio elege maior bancada policial de sua histór...,Tue Oct 09 18:00:01 +0000 2018,2,folha
1,1046251157025423360,fiquei tão triste quando eu vi o preço da câme...,Sun Sep 30 04:11:28 +0000 2018,0,:(
2,1041744620206653440,"Para Theresa May, seu plano para o Brexit é a ...",Mon Sep 17 17:44:06 +0000 2018,2,exame
3,1046937084727107589,caralho eu quero proteger a danielly em um pot...,Tue Oct 02 01:37:06 +0000 2018,0,:(
4,1047326854229778432,@SiCaetano_ viva o caos :),Wed Oct 03 03:25:55 +0000 2018,1,:)


## 2. Pré-processamento e transformações


---

**Tratamento de variáveis**

- Criação da variável **filtered_words**, onde o texto original (tweet_text) será submetido a tratamentos para definir as palavras chaves de sentimento para melhorar a classificação do modelo;<br>


- Criação da variável **join_f_words**, para concatenar as palavras do coluna **filtered_words**;<br>


- Criação da variável **normalize_words**, para normalizar a coluna **join_f_words** utilizando a biblioteca **enelvo**;<br>


- Exclusão de variáveis que entendemos não ser relevante para o modelo<br>
    **id**: ID único para o tweet<br>
    **tweet_date**: Data da publicação no Twitter<br>
    **query_used**: Filtro utilizado para buscar a publicação<br>


- Exclusão de variáveis tratadas<br>
    **tweet_text**: Texto da publicação no Twitter<br>



In [21]:
%%time

# O tratamendo dos tweets leva em média 10 min, em uma máquina i7 com 12 cores e 32 ram 

# Cria uma nova variável utilizando funções para tratamento das palavras do texto
df['filtered_words'] = df['tweet_text'].apply(lambda w: nlp_tratar_texto(w))


Wall time: 8min 37s


In [22]:
# Cria uma nova variável concatenando os tokens gerados pelo tratamento das palavras gerando um novo texto reduzido

df['join_f_words'] = df['filtered_words'].apply(lambda w: ' '.join(w))


In [23]:
%%time

# O tratamendo dos tweets utilizando o biblioteca enelvo leva em média 4 horas, em uma máquina i7 com 12 cores e 32 ram 

# Cria uma nova variável normalizando os tokens do novo texto reduzido

#df['normalize_words'] = df['join_f_words'].apply(lambda w: nlp_normalizar_texto(w))
#df['normalize_words'] = df['tweet_text'].apply(lambda w: nlp_tratar_texto(w, normalize=True))


Wall time: 0 ns


In [24]:
# Altera o tamanho da visualização das colunas para demonstrar todo o conteúdo
pd.set_option("display.max_colwidth", -1)

# Mostra o conteúdo das variáveis tratadas
#df[['tweet_text', 'filtered_words', 'join_f_words', 'normalize_words']]
df[['tweet_text', 'filtered_words', 'join_f_words']]


,tweet_text,filtered_words,join_f_words
0,Rio elege maior bancada policial de sua história https://t.co/sGXnhZKrHx https://t.co/Mcgiz70jPF,"[rio, eleger, maior, bancada, policial, história]",rio eleger maior bancada policial história
1,fiquei tão triste quando eu vi o preço da câmera :((((,"[ficar, triste, vir, preço, câmera]",ficar triste vir preço câmera
2,"Para Theresa May, seu plano para o Brexit é a única opção https://t.co/epl39YD9bj","[theresa, may, plano, brexit, única, opção]",theresa may plano brexit única opção
3,caralho eu quero proteger a danielly em um pote tadinhaa :(,"[caralho, querer, proteger, danielly, pote, tadinhaa]",caralho querer proteger danielly pote tadinhaa
4,@SiCaetano_ viva o caos :),"[vivo, caos]",vivo caos
...,...,...,...
94995,"Cuba e defensor de direitos humanos se unem contra chefe da OEA, intervenção militar na Venezuela. https://t.co/Lo2RvIgFBA https://t.co/CiPddCuRvw","[cuba, defensor, direitos, humanos, unir, contra, chefe, oea, intervenção, militar, venezuela]",cuba defensor direitos humanos unir contra chefe oea intervenção militar venezuela
94996,"#Oportunidade ➡️ Venha fazer parte da nossa equipe! Vagas abertas para alunos de Administração. Envie seu currículo e comprovante de matrícula para análise prévia até às 12h do dia 24/08, para o e-mail: vagasestagio@sistemafieto.com.br 😉 https://t.co/qvdFYeJh9I","[oportunidade, vir, fazer, parte, equipe, vagas, aberto, alunos, administração, enviar, currículo, comprovante, matrícula, análise, prévia, dia, mail]",oportunidade vir fazer parte equipe vagas aberto alunos administração enviar currículo comprovante matrícula análise prévia dia mail
94997,"@96syoo EU SEI 😭😭 é por isso que significa muito!! To feliz demais, eu amo ela :( e aqui da pra ver que ela deixa a bandeira na frente do palco e sai correndo pra pegar garrafa de água mas depois disso ela pegou de novo 😂 https://t.co/82oPAXYVNC","[saber, significar, to, feliz, demais, amar, aqui, ver, deixar, bandeira, frente, palco, sair, correr, pegar, garrafa, água, disso, pegar, novo]",saber significar to feliz demais amar aqui ver deixar bandeira frente palco sair correr pegar garrafa água disso pegar novo
94998,@louistsexhes N te conheço mas posta :D,"[n, conhecer, posto, d]",n conhecer posto d



---

**Conclusões**

Submetemos a variável tweet_text ao processamento de texto:

- Remoção de endereços de sites;
- Remoção de e-mails e usuários do tweet;
- Remoção de caracteres que não são letras;
- Remoção de dígitos;
- Transformação de todas as palavras para minúsculas;
- Tokenização do texto;
- Remoção de stopwords (portugues);
- "Lemmatização" do texto;

Após o tratamento do texto foram criadas duas variáveis para gravar as informações do texto tratado, entendemos que estas variáveis estejam mais limpas para utilização na modelagem.

- **join_f_words** que contem o texto tratado com o processamento descrito acima;
- **normalize_words** que contém o texto tratado e a normalização de texto utilizando a biblioteca **enelvo** que "corrige" abreviações, gírias, erros ortográficos e acrônimos;



In [25]:
#Grava um novo CSV com as novas colunas para avaliação

# Definindo o nome do arquivo
file_name = './dados/train/Train3Classes_with_Tokens.csv'
  
# Salvando o CSV
try:
    df.to_csv(file_name, index=False)
    print('DataFrame gravado com successo.')
except:
    print("Ocorreu um erro na gravação.")


DataFrame gravado com successo.


In [26]:
# Para executar esta celula descomentar o comando **#rollback = 1** para restaurar o df após tratamento de texto,
# para testes de repetição, devido ao tempo de normalização do texto

#rollback = 1

# Restaurar o dataframe df
if rollback == 1:
    # Carregando csv
    try:
        df = pd.read_csv('./dados/train/Train3Classes_with_Tokens.csv')
        
        DeleteList=['Unnamed: 0']
        df = df.drop(DeleteList, axis=1).copy()
        
        print('Arquivo carregado com successo.')
        rollback = 0
    except:
        print('Ocorreu um erro no carregamento do arquivo.')
else:
    print('Processo de rollback do dataframe não executado.')

Processo de rollback do dataframe não executado.


In [27]:
# Verificando quais linhas não apresentam valor após o tratamento do texto

df[np.where((df['join_f_words'].str.len()<1), True, False)]


,id,tweet_text,tweet_date,sentiment,query_used,filtered_words,join_f_words
215,1046982734827200513,Não mais :(( https://t.co/3omrxaGTJc,Tue Oct 02 04:38:30 +0000 2018,0,:(,[],
286,1049147450613792768,@yoongxsmin Eu também :(,Mon Oct 08 04:00:19 +0000 2018,0,:(,[],
476,1046999889597616133,@brooklenbarbie eu também :(,Tue Oct 02 05:46:40 +0000 2018,0,:(,[],
592,1046276432241004544,"@ywkheiz 1,57 só :( e a sua?",Sun Sep 30 05:51:54 +0000 2018,0,:(,[],
862,1047533065252327425,@b1ktwpas E é mesmo! : ),Wed Oct 03 17:05:19 +0000 2018,1,:),[],
...,...,...,...,...,...,...,...
93081,1047538834462969856,https://t.co/iHSV24IOgV,Wed Oct 03 17:28:15 +0000 2018,2,veja,[],
93294,1049253272220061696,@charroeciencia Que foi :(,Mon Oct 08 11:00:48 +0000 2018,0,:(,[],
94127,1049296782298173441,@JoaoGSimoes A minha e a tua :)),Mon Oct 08 13:53:42 +0000 2018,1,:),[],
94206,1045413174181343232,e foi :),Thu Sep 27 20:41:38 +0000 2018,1,:),[],


In [28]:
# Criando novo dataframe retirando as linhas que ficaram sem texto

# Novo dataframe dfc (df copy)
dfc = df[np.where((df['join_f_words'].str.len()>1), True, False)].copy()

# Reindexando os indices do dataframe
dfc.reset_index(drop=True, inplace=True)


In [29]:
# revisar o momento dessa execução

# Exclusão de variáveis que entendemos não ser relevante para o modelo

# id: ID único para o tweet  
# tweet_text: Texto da publicação no Twitter  
# tweet_date: Data da publicação no Twitter  
# query_used: Filtro utilizado para buscar a publicação
# filtered_words: Texto da publicação no Twitter após tratamento de texto

# Excluindo colunas
#DeleteList=['id', 'tweet_text', 'tweet_date', 'query_used', 'filtered_words']
DeleteList=['id', 'tweet_date', 'query_used', 'filtered_words']

# Novo dataframe dfc (df copy)
dfc = dfc.drop(DeleteList, axis=1).copy()


In [30]:
# Informações do dataframe

dfc.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94604 entries, 0 to 94603
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tweet_text    94604 non-null  object
 1   sentiment     94604 non-null  int64 
 2   join_f_words  94604 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [31]:
# Avaliando os valores nulos do dataframe

missing_values_table(dfc)


O dataframe tem 3 colunas e 94604 linhas.
Existem 0 colunas que têm valores faltantes.


,Valores faltantes,% de Valores Totais,Data Type



---

**Conclusões:** 

O dataframe é composto por 5 colunas e 94.604 registros.

A tabela acima apresenta valores faltantes, porém como não representa um percentual consideravél optamos em excluir os dados faltantes.


In [32]:
# Exclusão dos registros faltantes

dfc.dropna() 
dfc.reset_index(drop=True, inplace=True)


In [33]:
# Quantidade de linhas e colunas

qtlc, qtcc = dfc.shape

# Quantidade de linhas duplicadas
qtd, _ = dfc[dfc.duplicated()].shape


print('Dataframe original - df')
print(f'Quantidade de linhas...........: {qtl}')
print(f'Quantidade de colunas..........: {qtc}')

print('\nDataframe tratado - dfc')
print(f'Quantidade de linhas...........: {qtlc}')
print(f'Quantidade de linhas duplicadas: {qtd}')
print(f'Quantidade de colunas..........: {qtcc}')

print (f'\nPercentual de registros retirados: {(100  * (qtl-qtlc) / qtl):.2f}%')


Dataframe original - df
Quantidade de linhas...........: 95000
Quantidade de colunas..........: 5

Dataframe tratado - dfc
Quantidade de linhas...........: 94604
Quantidade de linhas duplicadas: 811
Quantidade de colunas..........: 3

Percentual de registros retirados: 0.42%


In [34]:
# Verificando linhas duplicadas

# Seleciona as linhas duplicadas no dataframe baseado em todas as colunas
display(dfc[dfc.duplicated(keep='first')].sort_values(by=list(dfc.columns)))

print("\nLinhas duplicadas:")
print(dfc[dfc.duplicated()].shape[0])


,tweet_text,sentiment,join_f_words
74022,"""Em recuperação judicial, Eternit vê receita crescer em agosto"" https://t.co/aKl26jNJyJ #trabalho #feedly",2,recuperação judicial eternit ver receita crescer agosto trabalho feedly
36418,"#CulturalSpam serale... #notizie #curiose, #interessanti, talvolta #utili! :D #curious, #interesting, sometimes #useful #news! :D Des #nouvelles #curieuses, #intéressantes, parfois #utiles! https://t.co/MjC5j2svqe :D",1,culturalspam serale notizie curiose interessanti talvolta utili d curious interesting sometimes useful news d des nouvelles curieuses intéressantes parfois utiles d
66554,#FollowFriday Todos os meus seguidores. :) #ff,1,followfriday todos seguidores ff
30159,#FollowFriday para todos meus queridos seguidores. :) #IFTTT,1,followfriday todos queridos seguidores ifttt
24782,"#Novidade Novo Kindle Oasis, tela de 7” sensível ao toque de alta resolução, à prova d’água, iluminação embutida, Wi-Fi https://t.co/kLVZK6xkBp Confira o preço no site. #kindle #amazon #leitor #ebook #KindleUnlimited #kindledeals #kindlebook",2,novidade novo kindle oasis tela sensível toque alta resolução prova d água iluminação embutir wi fi conferir preço site kindle amazon leitor ebook kindleunlimited kindledeals kindlebook
...,...,...,...
39077,tédio :(,0,tédio
55345,us2 q uminom :(((,0,q uminom
39161,voltei :),1,voltar
88831,vou tentar dormir :(,0,ir tentar dormir



Linhas duplicadas:
811


---

**Conclusões:** 

Após tratamento do dataframe identificamos que diversos texto possuem as mesmas palavras e como o percentual de duplicidade é baixo optamos em retirar as linhas repetidas.


In [35]:
# Preparando os dataframes

# Criando copia do dataframe tratado para manter todos os registros 
dfc_copy = dfc.copy()

# Convertendo int 64 para 32
dfc['sentiment'] = dfc['sentiment'].astype(np.int16)

# Excluindo os linhas duplicadas
dfc = dfc.drop_duplicates(keep='first').copy()
dfc.reset_index(drop=True, inplace=True)


In [36]:
# Quantidade de linhas e colunas

qtlc, qtcc = dfc.shape

# Quantidade de linhas duplicadas
qtd, _ = dfc[dfc.duplicated()].shape


print('Dataframe original - df')
print(f'Quantidade de linhas...........: {qtl}')
print(f'Quantidade de colunas..........: {qtc}')

print('\nDataframe tratado - dfc')
print(f'Quantidade de linhas...........: {qtlc}')
print(f'Quantidade de linhas duplicadas: {qtd}')
print(f'Quantidade de colunas..........: {qtcc}')

print (f'\nPercentual de registros retirados: {(100  * (qtl-qtlc) / qtl):.2f}%')


Dataframe original - df
Quantidade de linhas...........: 95000
Quantidade de colunas..........: 5

Dataframe tratado - dfc
Quantidade de linhas...........: 93793
Quantidade de linhas duplicadas: 0
Quantidade de colunas..........: 3

Percentual de registros retirados: 1.27%


---

**Conclusões:** 

- Tratamos a variável **tweet_text** para criar a variável auxiliar **filtered_words**, onde os textos foram submetidos a tratamentos para definir as palavras chaves de sentimento;<br>


- Criamos a variável **join_f_words**, para concatenar as palavras do coluna **filtered_words**;<br>


- Criamos a variável **normalize_words**, tratando a variável **tweet_text** utilizando a biblioteca **enelvo**;<br>


- Criamos um novo dataframe **dfc**, retirando as variáveis **id, tweet_date, query_used**, pois  as variáveis **id, tweet_date, query_used** não são relevantes para a modelagem;<br>


- Retiramos todos os registros que estavam duplicados;<br>


Com esses tratamentos acreditamos que o novo dataframe esta melhor preparado para análise exploratória de dados e modelagem.


## 3. Análise exploratória

In [ ]:
# Informações do dataframe

dfc.info()


In [ ]:
# Listagem das primeiras linhas do dataframe

dfc.head()


### Analisando Sentimentos

**sentiment**
- sentiment: 0, se negativo; 1, se positivo; 2, se neutro

In [ ]:
# Quantidade de sentimentos

Counter(dfc['sentiment'])


In [ ]:
# Distribuição de tweets

temp = dfc.groupby('sentiment').count()['join_f_words'].reset_index().sort_values(by='join_f_words',ascending=False)
temp.style.background_gradient(cmap=cmap_colors)


In [ ]:
# Distribuição da variavel sentiment

col = 'sentiment'

total = len(dfc)

plt.figure(figsize=(12,6))
g = sns.countplot(x=col, data=dfc, palette=colors)
g.set_title(f"Distribuição da variável {col}")
g.set_xlabel(f"{col}")
g.set_ylabel("Quantidade")
sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format((height/total)*100),
            ha="center", fontsize=14) 
g.set_ylim(0, max(sizes) * 1.15)
plt.show()


---

**Conclusões:** 

Analisando a variável **sentiment** podemos constatar que o nosso dataframe contém quantidades de tweets balanceados, diminuindo a tendência do modelo para algum dos resultados.


### Analisando as palavras dos textos

In [ ]:
# Palavras mais utilizadas nos textos

dfc['list_words'] = dfc['join_f_words'].apply(lambda w:str(w).split())

top = Counter([item for sublist in dfc['list_words'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Palavras','Quantidade']
temp.style.background_gradient(cmap=cmap_colors)


In [ ]:
# Analisando a quantidade de palavras mais utilizadas

plt.figure(figsize=(12,8))

g = sns.barplot(x='Quantidade', y='Palavras', data=temp, palette=colors)
g.set_title(f"As 20 palavras mais utilizadas nos textos")
g.set_xlabel(f"Quantidade")
g.set_ylabel("Palavras")
plt.show()


In [ ]:
# Analisando a quantidade de palavras mais utilizadas

fig = px.treemap(temp, path=['Palavras'], values='Quantidade',title='As 20 palavras mais utilizadas nos textos')
fig.show()


### Palavras mais utilizadas por Sentimento

In [ ]:
# Criando dataframes por sentimentos

# negativo
Negative_sent = dfc[dfc['sentiment']==0]
Negative_sent.reset_index(drop=True, inplace=True)

# positivo
Positive_sent = dfc[dfc['sentiment']==1]
Positive_sent.reset_index(drop=True, inplace=True)

# neutro
Neutral_sent = dfc[dfc['sentiment']==2]
Neutral_sent.reset_index(drop=True, inplace=True)


In [ ]:
# Lista auxiliar com todas as palavras do dataframe

raw_text = [word for word_list in dfc['list_words'] for word in word_list]


In [ ]:
%%time
# O processamento de cada instrução leva em média 7 minutos, totalizando aproximadamente 21 minutos
# em uma máquina i7 com 12 cores e 32 ram

# As 20 palavras utilizadas apenas para o sentimento negativo
Unique_Negative= words_unique(0, 20, raw_text)

# As 20 palavras utilizadas apenas para o sentimento positivo
Unique_Positive = words_unique(1, 20, raw_text)

# As 20 palavras utilizadas apenas para o sentimento neutro
Unique_Neutral= words_unique(2, 20, raw_text)


#### Sentimentos Negativos

In [ ]:
# Palavras mais utilizadas em sentimentos negativos

top = Counter([item for sublist in Negative_sent['list_words'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(20))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Palavras','Quantidade']
temp_negative.style.background_gradient(cmap='Reds')


In [ ]:
# Apresentação da quantidade de palavras mais utilizadas em sentimentos negativos

plt.figure(figsize=(12,8))

g = sns.barplot(x='Quantidade', y='Palavras', data=temp_negative, palette=colors)
g.set_title(f"Palavras mais utilizadas nos textos negativos")
g.set_xlabel(f"Quantidade")
g.set_ylabel("Palavras")
plt.show()


In [ ]:
# Apresenta as palavras utilizadas apenas em sentimentos negativos

print('As 20 palavras mais utilizadas apenas em Tweets negativos:')
Unique_Negative.style.background_gradient(cmap='Reds')


In [ ]:
# As 20 palavras utilizadas apenas para o sentimento negativos

fig = px.treemap(Unique_Negative, path=['words'], values='count',title='As 20 palavras mais utilizadas apenas em Tweets negativos')
fig.show()


#### Sentimentos Positivos

In [ ]:
# Palavras mais utilizadas em sentimentos positivos

top = Counter([item for sublist in Positive_sent['list_words'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['Palavras','Quantidade']
temp_positive.style.background_gradient(cmap=cmap_colors)


In [ ]:
# Apresentação da quantidade de palavras mais utilizadas em sentimentos positivos

plt.figure(figsize=(12,8))

g = sns.barplot(x='Quantidade', y='Palavras', data=temp_positive, palette=colors)
g.set_title(f"Palavras mais utilizadas nos textos positivos")
g.set_xlabel(f"Quantidade")
g.set_ylabel("Palavras")
plt.show()


In [ ]:
# Apresenta as palavras utilizadas apenas em sentimentos positivos

print('As 20 palavras mais utilizadas apenas em Tweets positivos:')
Unique_Positive.style.background_gradient(cmap=cmap_colors)


In [ ]:
# Apresentação das palavras utilizadas em sentimentos positivos

fig = px.treemap(Unique_Positive, path=['words'], values='count',title='As 20 palavras mais utilizadas apenas em Tweets positivos')
fig.show()


#### Sentimentos Neutros

In [ ]:
# Palavras mais utilizadas em sentimentos neutros

top = Counter([item for sublist in Neutral_sent['list_words'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Palavras','Quantidade']
temp_neutral.style.background_gradient(cmap=cmap_colors)


In [ ]:
# Apresentação da quantidade de palavras mais utilizadas em sentimentos neutros

plt.figure(figsize=(12,8))

g = sns.barplot(x='Quantidade', y='Palavras', data=temp_neutral, palette=colors)
g.set_title(f"Palavras mais utilizadas nos textos neutros")
g.set_xlabel(f"Quantidade")
g.set_ylabel("Palavras")
plt.show()


In [ ]:
# Apresenta as palavras utilizadas apenas em sentimentos neutros

print('As 20 palavras utilizadas apenas em Tweets neutros:')
Unique_Neutral.style.background_gradient(cmap=cmap_colors)


In [ ]:
# Apresentação das palavras utilizadas em sentimentos neutros

fig = px.treemap(Unique_Neutral, path=['words'], values='count',title='As 20 palavras mais utilizadas apenas em Tweets neutros')
fig.show()


#### WordClouds

In [ ]:
# WordCloud para Tweets negativos

text = ' '.join(Negative_sent['join_f_words'])
plot_wordcloud(text, title = 'WordCloud Tweets negativos', backcolor = 'white', clrmap = 'Reds')


In [ ]:
# WordCloud para Tweets positivos

text = ' '.join(Positive_sent['join_f_words'])
plot_wordcloud(text, title = 'WordCloud Tweets positivos', backcolor = 'white', clrmap = 'Greens')


In [ ]:
# WordCloud para Tweets neutros

text = ' '.join(Neutral_sent['join_f_words'])
plot_wordcloud(text, title = 'WordCloud Tweets neutros', backcolor = 'white', clrmap = 'PuBu')


---

**Conclusões:** 

- Podemos ver que as mesmas palavras são comuns nos três segmentos;


- Isso é interessante porque palavras como dont e cant são mais de natureza negativa e palavras como lol são mais de natureza positiva, o que significa que os nossos dados estão incorrectamente etiquetados, teremos mais informações sobre isso após a análise de N-grama;


- Olhando para as palavras únicas de cada sentimento,temos agora muito mais clareza sobre os dados, estas palavras únicas são determinantes para o sentimento dos tweets;


## 4. Treinamento do modelo

### Separando conjunto de dados Treino e Teste

In [ ]:
dfc['Num_words_text'] = dfc['join_f_words'].apply(lambda x:len(str(x).split()))

In [ ]:
# A nossa base de dados tem 93.793 mil linhas e a quantidade de palavras disponíveis nos textos é muito grande. 
# Para evitar problemas de alocação de memória e processamento dos textos e modelagem, iremos criar uma amostra com 70% da base:

dfc = dfc.sample(frac=0.7, replace=False)


In [ ]:
# Vamos dividir os nossos dados numa matriz X que contém as características a treinar, 
# e uma matriz y com a variável alvo, neste caso a coluna covid_res. 

X = dfc['join_f_words']
y = dfc['sentiment']


In [ ]:
# Vamos dividir os dados num conjunto de Train e num conjunto de Test. 
# Iremos treinar o modelo no conjunto de treino e depois utilizaremos o conjunto de testes para avaliar o modelo

random_seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed)


In [ ]:
# Quantidade total da variável "target" (sentiment)

y.value_counts().sort_index()


In [ ]:
# Quantidade separada para o conjunto de treino inicial

y_train.value_counts().sort_index()


In [ ]:
# Quantidade separada para o conjunto de teste inicial

y_test.value_counts().sort_index()#(normalize = True)


### Bag of Words

#### CountVectorizer

In [ ]:
# Instanciando CountVectorizer

cv = CountVectorizer()


In [ ]:
# Treinando o modelo com os conjuntos de dados de treinamento 

X_train_cv = cv.fit_transform(X_train).toarray()
X_test_cv = cv.transform(X_test).toarray()


In [ ]:
# Lista de modelos

list_models = [
{'model_name': 'XGBClassifier',
 'estimator' : XGBClassifier(n_jobs = 1, objective = 'multi:softmax', tree_method='approx', random_state = random_seed)},
{'model_name': 'XGBClassifier',
 'estimator' : XGBClassifier(n_jobs = 1, tree_method='approx', metric='multiclass', eval_metric='mlogloss', random_state = random_seed)}
#{'model_name': 'Logistic Regression',
# 'estimator' : LogisticRegression(random_state = random_seed, solver = 'lbfgs')}
#{'model_name': 'Decision Tree',
# 'estimator' : DecisionTreeClassifier(random_state = random_seed)}
#,{'model_name': 'Random Forest',
# 'estimator' : RandomForestClassifier(random_state = random_seed)}
#,{'model_name': 'AdaBoost',
# 'estimator' : AdaBoostClassifier(random_state = random_seed)}
]


In [ ]:
%%time

# Processando os modelos baseado na list_models

test_models_plot_roc_auc_curve(list_models,
                              'model_name',
                              'estimator',
                              X_train_cv,
                              X_test_cv,
                              y_train,
                              y_test)


#### TF-IDF

In [ ]:
#Instanciando TF-IDF

tfidf = TfidfVectorizer(use_idf = True)


In [ ]:
# Treinando o modelo com os conjuntos de dados de treinamento 

X_train_tfidf = tfidf.fit_transform(X_train).todense()
X_test_tfidf  = tfidf.transform(X_test).todense()


In [ ]:
# Processando os modelos baseado na list_models

test_models_plot_roc_auc_curve(list_models,
                               'model_name',
                               'estimator',
                               X_train_tfidf,
                               X_test_tfidf,
                               y_train,
                               y_test)


## 5. Conclusões